# Model Initiation

In [1]:
import sys

from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Load the Necessary API Keys

In [2]:
from google.colab import userdata

huggingface_api_key = userdata.get('hugging_face_key')
pinecone_key = userdata.get('pinecone_api_key')
mongo_uri = userdata.get('mongo_db_key')
open_ai_key = userdata.get('open_ai_api_key')


# Set-up/Connect to Pinecone

In [4]:
#create index
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=pinecone_key)
index_name = "rag-app-images"


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=512,
        metric="cosine",
        spec=ServerlessSpec(cloud = "aws", region="us-east-1")
    )

index = pc.Index(index_name)

#vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [5]:
from huggingface_hub import login
from huggingface_hub import whoami

login(huggingface_api_key)
whoami()

{'type': 'user',
 'id': '66d5147ab005ad82ca47182f',
 'name': 'dorukozar',
 'fullname': 'Doruk Ozar',
 'email': 'dorukozar@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'periodEnd': None,
 'isPro': False,
 'avatarUrl': '/avatars/06335824f9a6991ec7b901b31802dd5b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Presentation',
   'role': 'read',
   'createdAt': '2025-01-16T00:00:59.134Z'}}}

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#create index
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=pinecone_key)
index_name = "rag-app"


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud = "aws", region="us-east-1")
    )

index_text = pc.Index(index_name)

vector_store_text = PineconeVectorStore(embedding=huggingface_embeddings, index=index_text)

In [9]:
user_query = "What is supervised and unsupervised machine learning?"
retriever = vector_store_text.as_retriever(search_kwargs={"k":5})
# retriever.get_relevant_documents(query)
retrieved_docs = retriever.invoke(user_query)
retrieved_docs

relevant_docs = vector_store_text.as_retriever(search_kwargs={"k":5}).invoke(user_query)

[Document(id='3fa65848-c8ee-43e9-8856-541127519cf5', metadata={'source_id': 'ML_Overview.txt'}, page_content="getting very specific into the models i mean there's there's another step right here it might be like semi-supervised or reinforcement but generally we have two really broad categories supervised and unsupervised the difference between these is very simple supervised has labels we know where the data comes from and we know what the target classes are and unsupervised does not all right and we're going to talk about this a little bit more detail here as i get into some examples but that's it i mean this is just categorized right this is not all right much of the advancements when we think of like in the ai systems or advances in technology all come from labeled data right so it's you have armies of intern somewhere labeling it whether it's a dog or a hot dog somewhere right we pass all those all those images in and a machine knows it or anything the same for ibm or the deep blue

In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print(format_docs(retrieved_docs))

getting very specific into the models i mean there's there's another step right here it might be like semi-supervised or reinforcement but generally we have two really broad categories supervised and unsupervised the difference between these is very simple supervised has labels we know where the data comes from and we know what the target classes are and unsupervised does not all right and we're going to talk about this a little bit more detail here as i get into some examples but that's it i mean this is just categorized right this is not all right much of the advancements when we think of like in the ai systems or advances in technology all come from labeled data right so it's you have armies of intern somewhere labeling it whether it's a dog or a hot dog somewhere right we pass all those all those images in and a machine knows it or anything the same for ibm or the deep blue or the other really famous uh kind of ai machines that uh that you think of right they all learn to play

of 

# Embed the query

In [11]:
inputs = clip_processor(text=[user_query], return_tensors="pt", padding=True)
text_embedding = clip_model.get_text_features(**inputs).detach().numpy().tolist()[0]

# Query the Pinecone vector db and return top 5 matches

In [12]:
query_results = index.query(vector=text_embedding, top_k=5, include_metadata=True)

In [13]:
query_results["matches"]

[{'id': '3001_ETA/page_9_img_2.png',
  'metadata': {'file_name': '3001_ETA', 'image_key': 'page_9_img_2.png'},
  'score': 0.334317118,
  'values': []},
 {'id': 'machine_learning_bootcamp_II/page_8_img_1.png',
  'metadata': {'file_name': 'machine_learning_bootcamp_II',
               'image_key': 'page_8_img_1.png'},
  'score': 0.322387844,
  'values': []},
 {'id': 'machine_learning_overview/page_8_img_1.png',
  'metadata': {'file_name': 'machine_learning_overview',
               'image_key': 'page_8_img_1.png'},
  'score': 0.322387844,
  'values': []},
 {'id': 'machine_learning_III/page_8_img_1.png',
  'metadata': {'file_name': 'machine_learning_III',
               'image_key': 'page_8_img_1.png'},
  'score': 0.322387844,
  'values': []},
 {'id': 'machine_learning_bootcamp_II copy/page_8_img_1.png',
  'metadata': {'file_name': 'machine_learning_bootcamp_II copy',
               'image_key': 'page_8_img_1.png'},
  'score': 0.322387844,
  'values': []}]

# Connect to Mongodb

In [15]:
from pymongo import MongoClient
import gridfs

client = MongoClient(mongo_uri)

#client = MongoClient(MONGO_URI)

db = client["images"]

collection = db["images_for_rag"]

print("Connected to MongoDB successfully!")

Connected to MongoDB successfully!


In [16]:
#file_names_list = os.listdir("data_processed")

fs = gridfs.GridFS(db)



# Query MongDB

In [17]:
#image_data = fs.find_one({"filename": "clustering.png"})
image_data_list = []
for i in query_results["matches"]:
    filename = i["id"]
    image_data = fs.find_one({"filename": filename})
    image_data_list.append(image_data)

In [18]:
image_data_list

In [19]:
from PIL import Image, ImageChops
import io
import hashlib
import numpy as np
from skimage.metrics import structural_similarity as ssim

def images_are_equal(img1, img2):
    """Check if two images are identical pixel by pixel."""
    return ImageChops.difference(img1, img2).getbbox() is None

def hash_image(image):
    """Compute hash of an image."""
    hasher = hashlib.md5()
    hasher.update(image.tobytes())  # Convert image to bytes and hash
    return hasher.hexdigest()

def images_are_similar(img1, img2, threshold=0.7):
    """Compare two images using SSIM after resizing them to the same dimensions."""

    # Convert to grayscale
    img1_gray = img1.convert('L')
    img2_gray = img2.convert('L')

    # Resize images to the same size
    common_size = (min(img1_gray.width, img2_gray.width), min(img1_gray.height, img2_gray.height))
    img1_resized = img1_gray.resize(common_size, Image.LANCZOS)
    img2_resized = img2_gray.resize(common_size, Image.LANCZOS)

    # Convert to NumPy arrays
    img1_np = np.array(img1_resized)
    img2_np = np.array(img2_resized)

    # Compute SSIM similarity
    similarity = ssim(img1_np, img2_np)
    print(similarity)
    return similarity > threshold  # Return True if similar

In [20]:
from PIL import Image, ImageChops
import io
import hashlib
import numpy as np
from skimage.metrics import structural_similarity as ssim



if len(image_data_list)>0:
    local_image_list = []
    hashes = set()
    user_input = input("""
    To use similarity, enter the number 1
    To use hashing comparison, enter the number 2
    To use pixel to pixel comparison, enter the number 3
    """)

    for j in image_data_list:
        if j:
            # Convert binary data to a PIL Image
            image = Image.open(io.BytesIO(j.read()))

            if user_input.strip() == "1":
                duplicate_found = False
                for stored_image in local_image_list:
                    if images_are_similar(stored_image, image):
                        print(f"⚠️ Similar image found at index {j}")
                        duplicate_found = True
                        break

                if not duplicate_found:
                    local_image_list.append(image)
                    image.show()

            elif user_input.strip() == "2":
                img_hash = hash_image(image)
                if img_hash in hashes:
                    print(f"⚠️ Duplicate image found at index {j}")
                else:
                    hashes.add(img_hash)
                    image.show()

            elif user_input.strip() == "3":
                duplicate_found = False
                for stored_image in local_image_list:
                    if images_are_equal(stored_image, image):
                        print(f"⚠️ Duplicate image found at index {j}")
                        duplicate_found = True
                        break

                if not duplicate_found:
                    local_image_list.append(image)
                    image.show()
            else:
                print("Wrong input!")
                sys.exit(1)


            # Display the image
            # image.show()
        else:
            print("❌ Image not found")





    To use similarity, enter the number 1
    To use hashing comparison, enter the number 2
    To use pixel to pixel comparison, enter the number 3
    3
⚠️ Duplicate image found at index <gridfs.synchronous.grid_file.GridOut object at 0x78422828c790>
⚠️ Duplicate image found at index <gridfs.synchronous.grid_file.GridOut object at 0x78422828fdc0>
⚠️ Duplicate image found at index <gridfs.synchronous.grid_file.GridOut object at 0x78422828fa60>


In [21]:
local_image_list

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1300x917>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1222x917>]

---
break
---

# Embedding text and image at the same time

In [22]:
import base64
from PIL import Image
import io


encoded_images = []
count = 0

if len(image_data_list) > 0:
    for j in local_image_list:
        if j:
            print(f"Processing image {count + 1}")

            try:
                # Ensure the file pointer is at the start
                j.seek(0)


                # Convert to PNG format and encode
                buffered = io.BytesIO()
                j.save(buffered, format="JPEG")

                encoded_image = base64.b64encode(buffered.getvalue()).decode("utf-8")
                encoded_images.append(encoded_image)

                count += 1
            except Exception as e:
                print(f"❌ Error processing image {count + 1}: {e}")
        else:
            print("❌ Image not found")

# Closing file handlers
for img in image_data_list:
    img.close()

# Displaying the number of successfully encoded images
print(f"✅ Successfully encoded {count} images.")


Processing image 1
Processing image 2
✅ Successfully encoded 2 images.


In [23]:
# prompt = f"""You are an expert LLM assistant specialized in answering questions related to computer science/data science/machine learning/LLM. Use the retrieved information from RAG (Retrieved information and Image Descriptions) and your knowledge to respond accurately and clearly to each question.
#
# Guidelines:
# 1. Provide concise and informative answers.
# 2. If the question is beyond the scope of your knowledge or the provided information, state, "I don't know."
# 3. If the context section (the information that is returned by RAG pipeline) has no information about a part of the question, please express that "The retrieved information did not contain answer to this question" but if you can answer it based on your knowledge please do
# 4. Use examples where applicable to illustrate your answers.
# 5. Maintain a professional and helpful tone.
#
# Question: {user_query}
#
# Retrieved Information: {format_docs(retrieved_docs)}
#
# Answer:
# """

prompt = f"""
Guidelines:
1. Provide extensive and informative answers.
2. If the question is beyond the scope of your knowledge or the provided information, state, "I don't know."
3. If the context section (the information that is returned by RAG pipeline) has no information about a part of the question, please express that "The retrieved information did not contain answer to this question" but if you can answer it based on your knowledge please do
4. Use examples where applicable to illustrate your answers.
5. Maintain a professional and helpful tone.

Question: {user_query}

Retrieved Information: {format_docs(retrieved_docs)}

Answer:
"""


In [24]:
content = [{"type":"text", "text":prompt}]
for img in encoded_images:
    content.append({
        "type":"image_url",
        "image_url":{
            "url":f"data:image/jpeg;base64, {img}"
        }
    })

In [27]:
from openai import OpenAI

client = OpenAI(api_key=open_ai_key)

try:
    chat = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
            "role":"system",
            "content":"You are an expert LLM assistant specialized in answering questions related to computer science/data science/machine learning/LLM. Use the retrieved information from RAG (Retrieved information and Image Descriptions) and your knowledge to respond accurately and clearly to each question."
        },
            {
                "role": "user",
                "content": content
            }
        ]
    )

    print(chat.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

Supervised and unsupervised machine learning are two fundamental types of learning algorithms used in artificial intelligence and data analysis.

### Supervised Machine Learning

**Definition:**
Supervised learning involves training a model using a dataset that contains both input data and the corresponding correct output. The model learns to map inputs to outputs based on this labeled training data.

**Purpose:**
The goal is to predict the output for new, unseen data. Supervised learning is often used for classification (predicting a category) and regression (predicting a continuous value) tasks.

**Examples:**
- **Classification:** Identifying emails as 'spam' or 'not spam.' The algorithm is trained on emails labeled as spam or not.
- **Regression:** Predicting house prices based on historical data of house prices and features like location, size, etc.

**How it Works:**
1. The model takes in input-output pairs from the training data.
2. It learns a function that maps input data to t

In [ ]:
# RAGAS EVALUATION

In [62]:
sample_queries = [
    "What are the components of a decision tree?",
    "What is k-means clustering?",
    "What is the bias-variance tradeoff?",
    "What is overfitting in the context of machine learning?",
    "What does a decision boundary mean in the context of K-nearest neighbors?",
    "What are ensemble learning methods, and how do they improve model performance?"
]

expected_responses = [
    "A decision tree consists of a root node, internal nodes, branches, and leaf nodes. The root and internal nodes represent decisions or tests on features, branches indicate possible outcomes, and leaf nodes contain final predictions or classifications.",
    "K-means clustering is an unsupervised learning algorithm that partitions data into kk clusters. It iteratively assigns points to the nearest cluster centroid, recalculates centroids, and repeats until convergence. It’s useful for grouping similar data points in tasks like image segmentation and customer segmentation.",
    "The bias-variance tradeoff describes the balance between underfitting and overfitting in a model. High bias leads to underfitting, where the model oversimplifies patterns, while high variance causes overfitting, capturing noise rather than true patterns. Optimal performance requires a balance between the two.",
    "Overfitting occurs when a model learns noise and details from the training data instead of general patterns, leading to poor generalization on new data. It happens when the model is too complex, with too many parameters relative to the dataset size.",
    "In K-nearest neighbors (KNN), a decision boundary is a region in feature space that separates different class labels. It is determined by the distribution of training points and varies based on the value of kk, with smaller kk leading to more complex boundaries.",
    "Ensemble learning combines multiple models to improve accuracy and robustness. Techniques like bagging, boosting, and stacking reduce overfitting, increase stability, and enhance predictions by leveraging the strengths of multiple weak learners, often outperforming individual models."
]



In [63]:
def generate_answer(query, relevant_docs):
    """Generate an answer for a given query based on the most relevant document."""
    prompt = f"question: {query}\n\nDocuments: {relevant_docs}"
    messages = [
        {"role": "system", "content": "You are a helpful assistant that answers questions based on given documents only."},
        {"role": "user", "content": prompt},
    ]
    try:
        chat = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return chat.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [67]:
dataset = []

for query,reference in zip(sample_queries,expected_responses):

    relevant_docs = [
    doc.page_content for doc in vector_store_text.as_retriever(search_kwargs={"k":5}).invoke(query)
]
    response = generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

In [68]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [72]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o", openai_api_key=open_ai_key))
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

{'context_recall': 0.8333, 'faithfulness': 0.8858, 'factual_correctness': 0.5767}

In [ ]:
# Synthetic Testset generation

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
from ragas.testset import TestsetGenerator

# Step 1: Retrieve Documents from Pinecone (Assuming you have stored them with metadata)
retrieved_docs = vector_store_text.similarity_search(query="Retrieve all docs", k=100)  # Adjust 'k' as needed

# Step 2: Convert Retrieved Data into LangChain Documents
formatted_docs = [
    Document(page_content=doc.page_content, metadata=doc.metadata) for doc in retrieved_docs
]

# Step 3: Initialize RAGAS Testset Generator
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# Step 4: Generate Dataset
dataset = generator.generate_with_langchain_docs(formatted_docs, testset_size=10)

print("Dataset generation complete.")


In [60]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How can I locate and copy the us murders data ...,[working directory.\nWe have included the us m...,To locate and copy the us murders data file us...,single_hop_specifc_query_synthesizer
1,How does MapReduce facilitate the building of ...,[search index such as Lucene works: it is a fi...,MapReduce facilitates the building of search i...,single_hop_specifc_query_synthesizer
2,what chapter 3 say bout index updates?,[| \n411Since querying a search index by keywo...,"Chapter 3 discusses that if you want to add, r...",single_hop_specifc_query_synthesizer
3,Wht is MongoDB's advntage in data storage?,[or a binary variant thereof (such as MongoDB’...,MongoDB offers a performance advantage in data...,single_hop_specifc_query_synthesizer
4,Wht is VoltDB and how is it used in partitione...,[document-partitioned index is also known as a...,VoltDB is a database system that is used in pa...,single_hop_specifc_query_synthesizer
5,How NoSQL databases like MongoDB use different...,"[<1-hop>\n\nqueries, but it can cope better if...","NoSQL databases, such as MongoDB, use differen...",multi_hop_specific_query_synthesizer
6,"How does the use of NoSQL databases, like Mong...","[<1-hop>\n\nqueries, but it can cope better if...","NoSQL databases, such as MongoDB and CouchDB, ...",multi_hop_specific_query_synthesizer
7,How does Google’s Spanner database utilize dat...,"[<1-hop>\n\ndocument-partitioned indexes, 207\...",Google’s Spanner database utilizes data locali...,multi_hop_specific_query_synthesizer
8,How graph-like data models help in managing hi...,[<1-hop>\n\ncation code and worse performance ...,Graph-like data models are particularly benefi...,multi_hop_specific_query_synthesizer
9,How has the field of data science evolved to a...,[<1-hop>\n\ninternational group of physicists ...,The field of data science has evolved signific...,multi_hop_specific_query_synthesizer
